In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import os 
import numpy as np
import torch 

from law import ScalingLaw

In [3]:
def load_losses(dirs, slice_list, task_name, filter_word=None, exclude_word=None):
    files_to_exclude = ['trace', 'val_inputs', 'labels', 'proportions', 'emb', 'rouge', 'generations', 'gradient', 'acc']    
    df_all = pd.DataFrame() 

    count = 0
    for dir in dirs:
        files = os.listdir(dir)
        files = [os.path.join(dir, f) for f in files]
        files.sort(key=lambda x: os.path.getmtime(x), reverse=True)

        for file in files:
            if ".log" in file or task_name not in file:
                continue 

            if any(skill not in file for skill in slice_list):
                continue 

            method = file.split("/")[-1]

            if filter_word is not None and filter_word not in method:
                continue

            if exclude_word is not None and exclude_word in method:
                continue
           
            if "stratified" in file or "doge" in file or "aioli" in file or "skillit" in file:
                continue 
            

            weight_str = method.split("weights_")[-1].split("_")[0]

            if len(weight_str) == 9:
                a = int(weight_str[0])
                b = int(weight_str[1])
                c = int(weight_str[2])
                d = int(weight_str[3])
                e = int(weight_str[4])
                f = int(weight_str[5])
                g = int(weight_str[6])
                h = int(weight_str[7])
                i = int(weight_str[8])
            else:
                a, b, c, d, e, f, g, h, i = [float(f"0.{weight}") for weight in weight_str.split("0.")[1:]]

            print(a, b, c, d, e, f, g, h, i)
            count += 1 

            runs = os.listdir(file)
            for run in runs:
                if "test_" in run:
                    continue

                if any([exclude_file in run for exclude_file in files_to_exclude]):
                    continue 

                seed = int(run.split("seed_")[-1].split("_")[0])
                checkpoint = int(run.split("-")[-1].split(".")[0])


                path = os.path.join(file, run)

                df = pd.read_pickle(path)
                df = df.rename(columns={"task_idx": "skill", "task_loss": "loss"})

                df["method"] = method
                df["seed"] = seed
                df["checkpoint"] = checkpoint
                df["p1"] = a 
                df["p2"] = b
                df["p3"] = c
                df["p4"] = d
                df["p5"] = e
                df["p6"] = f
                df["p7"] = g
                df["p8"] = h
                df["p9"] = i


                df.set_index("checkpoint", inplace=True)

                df_all = pd.concat([df_all, df])

    df_all = df_all.sort_values(by=["checkpoint", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "seed"])
    return df_all


In [4]:
def mixing_law(x, param):
    log_c_i, b_i = param[0], param[1]
    t_i = param[2:]
    result = torch.exp(log_c_i) + torch.exp(b_i + torch.matmul(x[:, :8], t_i))
    return result

def init_params_law(idx, num_domains=9):
    for log_c_i in np.linspace(-2, 1.5, 10):
        for b_i in np.linspace(-10, 1, 20):
            for _ in range(30):
                ts = [-np.random.rand() if i == idx else np.random.rand() * 0.1 for i in range(num_domains-1)]
                yield [log_c_i, b_i] + ts

In [5]:
def mixing_law_diagonal(x, param):
    log_c_i, b_i, t_i = param[0], param[1], param[2]
    result = torch.exp(log_c_i) + torch.exp(b_i + x*t_i)
    return result

def init_params_law_diagonal():
    for log_c_i in np.linspace(-2, 1.5, 10):
        for b_i in np.linspace(-10, 1, 20):
            for _ in range(30):
                ts = [-np.random.rand()]
                yield [log_c_i, b_i] + ts

In [6]:
def calculate_r_squared(actuals, predictions):
    actuals, predictions = actuals.numpy(), predictions.numpy()
    # Calculate the total sum of squares
    total_sum_of_squares = np.sum((actuals - np.mean(actuals)) ** 2)
    # Calculate the residual sum of squares
    residual_sum_of_squares = np.sum((actuals - predictions) ** 2)
    # Calculate R-squared
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    return r_squared


# Instruction tuning

In [ ]:
dirs = ["../output/10282024/", "../output/10292024/"] # REPLACE WITH YOUR RUN OUTPUT DIRECTORIES
task_name = "instruction"
slice_list = [""]
df = load_losses(dirs, slice_list, task_name)

In [8]:
grid = 50
all_weights = []
for seed in range(5):
    with open(f"../dirichlet_weights/k_9_n_{grid}_seed_{seed}.txt", "r") as f:
        all_weights.extend(f.readlines())

all_weights = [weight_line.strip().replace(",", "") for weight_line in all_weights]
all_weights = [weight_line.rstrip("0") for weight_line in all_weights]

while "00." in "".join(all_weights):
    all_weights = [weight_line.replace("00.", "0.") for weight_line in all_weights]

df = df[df.method.str.contains("|".join(all_weights))]


In [11]:
skills = sorted(df.skill.unique())
indices = [1000] # end of training
seeds = sorted(df.seed.unique())

In [ ]:
params = {skill : {idx: {seed: {} for seed in seeds} for idx in indices} for skill in skills }
mses = []
r2s = []

for i, skill in enumerate(skills):
    for index in indices:
        for seed in seeds:
            print(f"Skill = {skill}, index = {index}, seed = {seed}")

            df_subset = df.loc[df.index == index]
            df_subset = df_subset[(df_subset.skill == skill) & (df_subset.seed == seed)]

            x = df_subset[['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']].values 

            y = df_subset['loss'].values
                
            law = ScalingLaw(mixing_law)

            p = law.fit(x, y, init_params_law(i, num_domains=len(skills)), max_step=100, delta=0.02)
            params[skill][index][seed] = p

            prediction_train = mixing_law(torch.tensor(x, dtype=torch.float), torch.tensor(p, dtype=torch.float))
            rmse_train = (torch.mean((prediction_train - y)**2)**0.5).item()
            mae_train = torch.mean(torch.abs(prediction_train - y)).item()
            mse_train = torch.nn.functional.mse_loss(prediction_train, torch.tensor(y, dtype=torch.float)).item()
            r2_train = calculate_r_squared(torch.tensor(y), torch.tensor(prediction_train))

            mses.append(mse_train)
            r2s.append(r2_train)


            print(f"RMSE: {rmse_train}, MAE: {mae_train}, MSE: {mse_train}, R2: {r2_train}")

mses = np.array(mses)
r2s = np.array(r2s)

print(mses.mean(), mses.std())
print(r2s.mean(), r2s.std())

In [14]:
import pickle 

with open("./law_results/instruction/params.pkl", "wb") as f:
    pickle.dump(params, f)